In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

### INDEXING

In [4]:
# Load Documents
doc_path = "dev-data/Be_Good.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# Embed
vectorestore = Chroma.from_documents(
    documents=splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)

retriever = vectorestore.as_retriever(search_kwargs={"k": 1})

### RETRIEVEAL and GENERATION

In [5]:
# Prompt 
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-preview-05-20")

# Post-processing 
def format_docs(docs):
    for doc in docs:
        print(doc.page_content, end="\n\n")
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("Who is the writter of the book")

C:\Users\Tommy RAMAROKOTO\Documents\programmation\projects\New folder\chatbots\backend\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


it's the only algorithm that works on that scale.When you write something telling people to be good,
you seem to be
claiming to be good yourself.  So I want to say explicitly that I
am not a particularly good person.  When I was a kid I was firmly
in the camp of bad.  The way adults used the word good, it seemed
to be synonymous with quiet, so I grew up very suspicious of it.You know how there are some
people whose names come up in conversation
and everyone says "He's such a great guy?"  People never say
that about me.  The best I get is "he means well."  I am not claiming
to be good.  At best I speak good as a second language.So I'm not suggesting you be good in the
usual sanctimonious way.
I'm suggesting it because it works.  It will work not just as a
statement of "values," but as a guide to strategy,
and even a design spec for software.  Don't just not be evil.  Be
Page 9



"I apologize, but the provided context does not contain information about the writer of the book. It only includes the author's personal reflections without mentioning their name."